<a href="https://colab.research.google.com/github/pranavi2013/2203a52118/blob/main/Lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
a=pd.read_csv('/content/train-1.csv')
a.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1
3,F00000005,257,1,1,1,0,0.0,0,2,1
4,F00000006,342,1,0,1,0,0.0,0,2,1


In [ ]:
a.shape

(88858, 10)

In [ ]:
b=pd.read_csv('/content/test.csv')
b.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season
0,F00000002,188,1,1,1,0,NaN,0,2
1,F00000007,410,1,1,1,0,0.0,0,2
2,F00000011,626,1,0,1,0,0.0,0,2
3,F00000013,731,1,0,1,0,0.0,0,2
4,F00000014,789,0,0,1,0,0.0,0,1


In [ ]:
b.shape

(59310, 9)

In [ ]:
nan_check = a.isna().any()
a = a.dropna(axis=0)
a.to_csv('cleaned_data.csv', index=False)

In [ ]:
df=a.drop('ID',axis=1)
df.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,188,1,0,1,0,0.0,0,1,0
1,209,1,0,1,0,0.0,0,2,1
2,257,1,0,1,0,0.0,0,2,1
3,257,1,1,1,0,0.0,0,2,1
4,342,1,0,1,0,0.0,0,2,1


In [ ]:
x=df.drop('Crop_Damage',axis=1)
print(x)
y=df['Crop_Damage']
print(y)

       Estimated_Insects_Count  Crop_Type  Soil_Type  Pesticide_Use_Category  \
0                          188          1          0                       1   
1                          209          1          0                       1   
2                          257          1          0                       1   
3                          257          1          1                       1   
4                          342          1          0                       1   
...                        ...        ...        ...                     ...   
88853                     3337          1          0                       2   
88854                     3516          1          0                       2   
88855                     3516          1          0                       2   
88856                     3702          1          0                       2   
88857                     3895          1          0                       2   

       Number_Doses_Week  Number_Weeks_

In [ ]:
scalar=MinMaxScaler()#normalized
scalar.fit(x)
x_scaled=scalar.transform(x)
print(x_scaled)

[[0.00962757 1.         0.         ... 0.         0.         0.        ]
 [0.01494806 1.         0.         ... 0.         0.         0.5       ]
 [0.0271092  1.         0.         ... 0.         0.         0.5       ]
 ...
 [0.85279959 1.         0.         ... 0.59701493 0.16       0.5       ]
 [0.89992399 1.         0.         ... 0.37313433 0.36       1.        ]
 [0.94882189 1.         0.         ... 0.55223881 0.14       1.        ]]


In [ ]:
from sklearn.model_selection import train_test_split
x_scaled_train, x_scaled_test, y_train, y_test = train_test_split(
    x_scaled, y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
svm_model = SVC()
svm_model.fit(x_scaled_train, y_train)
svm_predictions = svm_model.predict(x_scaled_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)
svm_classification_report = classification_report(y_test, svm_predictions)
print("SVM Classification Report:\n", svm_classification_report)
svm_confusion_matrix = confusion_matrix(y_test, svm_predictions)
print("SVM Confusion Matrix:\n", svm_confusion_matrix)
n_bootstrap_samples = 1000
bootstrap_accuracies = []
for _ in range(n_bootstrap_samples):
    x_sample, y_sample = resample(x_scaled_train, y_train, replace=True, random_state=10)
    svm_model.fit(x_sample, y_sample)
    svm_sample_predictions = svm_model.predict(x_scaled_test)
    accuracy = accuracy_score(y_test, svm_sample_predictions)
    bootstrap_accuracies.append(accuracy)
bootstrap_accuracies.sort()
lower_percentile = 2.5
upper_percentile = 97.5
lower_bound = bootstrap_accuracies[int((lower_percentile / 100) * n_bootstrap_samples)]
upper_bound = bootstrap_accuracies[int((upper_percentile / 100) * n_bootstrap_samples)]
print("Bootstrap Accuracy (95% Confidence Interval):", lower_bound, "-", upper_bound)


SVM Accuracy: 0.8424117205108941
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.84      1.00      0.91     13383
           1       0.64      0.04      0.08      2194
           2       0.00      0.00      0.00       395

    accuracy                           0.84     15972
   macro avg       0.50      0.35      0.33     15972
weighted avg       0.80      0.84      0.78     15972

SVM Confusion Matrix:
 [[13363    20     0]
 [ 2102    92     0]
 [  364    31     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(x_scaled_train, y_train)
knn_predictions = knn_model.predict(x_scaled_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("KNN Accuracy:", knn_accuracy)
